## IBM Professional Certificate Data Science

### Peer graded assignement - TORONTO neighborhood segmentation and clustering

This notebook will be used to undertake work for this project and is divided in 3 parts (for the 3 assignment questions).

##  

## Part 1 - Scraping Toronto neighboorhood information

Loading libraries

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scraping wikipedia page

In [3]:
webpage_response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(webpage_response.content, "html.parser")
#print(soup)

In [4]:
# Find the neighborhood table and generate a list of all the text entries
table = soup.find('table',{'class':'wikitable'})
td = table.find_all('td')
list_txt = []
for i in td:
    txt = i.get_text()
    list_txt.append(txt)
print(list_txt)


['M1A\n', 'Not assigned\n', '\n', 'M2A\n', 'Not assigned\n', '\n', 'M3A\n', 'North York\n', 'Parkwoods\n', 'M4A\n', 'North York\n', 'Victoria Village\n', 'M5A\n', 'Downtown Toronto\n', 'Regent Park / Harbourfront\n', 'M6A\n', 'North York\n', 'Lawrence Manor / Lawrence Heights\n', 'M7A\n', 'Downtown Toronto\n', "Queen's Park / Ontario Provincial Government\n", 'M8A\n', 'Not assigned\n', '\n', 'M9A\n', 'Etobicoke\n', 'Islington Avenue\n', 'M1B\n', 'Scarborough\n', 'Malvern / Rouge\n', 'M2B\n', 'Not assigned\n', '\n', 'M3B\n', 'North York\n', 'Don Mills\n', 'M4B\n', 'East York\n', 'Parkview Hill / Woodbine Gardens\n', 'M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n', 'M6B\n', 'North York\n', 'Glencairn\n', 'M7B\n', 'Not assigned\n', '\n', 'M8B\n', 'Not assigned\n', '\n', 'M9B\n', 'Etobicoke\n', 'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale\n', 'M1C\n', 'Scarborough\n', 'Rouge Hill / Port Union / Highland Creek\n', 'M2C\n', 'Not assigned\n', '\n', '

Building the dataframe from the scrape data

In [5]:
# Extracting text into lists for postcodes, boroughs and neighborhoods
postcodes = list_txt[::3]
boroughs = list_txt[1::3]
neighborhoods = list_txt[2::3]

# Creating the dataframe from these lists
df= pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df['PostalCode'] = pd.Series(postcodes)
df['Borough'] = pd.Series(boroughs)
df['Neighborhood'] = pd.Series(neighborhoods)

# Cleaning the dataframe
df.replace(regex='\n',value='', inplace=True)
df.replace(regex=' / ',value=' , ', inplace=True)
df = df[df['Borough'] != 'Not assigned']

In [6]:
# Search for rows with unassigned neighboords -> there are none
df_s = df[df['Neighborhood']=='Not assigned']
df_s

,PostalCode,Borough,Neighborhood


In [7]:
# Display the dataframe
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [8]:
# Display the number of rows
print('The Toronto neighborhood dataframe consists of {} rows'.format(df.shape[0]))

The Toronto neighborhood dataframe consists of 103 rows


##  

## Part 2- Insert geospatial data

Loading additional libraries

In [9]:
#!pip install geocoder
#!pip install pgeocode
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pgeocode
import geocoder # import geocoder

Fetching the geospatial data with pgeocode python app

In [10]:
nomi = pgeocode.Nominatim('ca')
nomi.query_postal_code("M4A")

df_gps = pd.DataFrame(columns=['PostalCode','Latitude','Longitude'])

for postalcode in df['PostalCode']:
    lat = nomi.query_postal_code(postalcode).latitude
    long = nomi.query_postal_code(postalcode).longitude
    df_gps = df_gps.append({'PostalCode':postalcode, 'Latitude':lat, 'Longitude':long}, ignore_index=True)
    
df_gps.head()

,PostalCode,Latitude,Longitude
0,M3A,43.7545,-79.3300
1,M4A,43.7276,-79.3148
2,M5A,43.6555,-79.3626
3,M6A,43.7223,-79.4504
4,M7A,43.6641,-79.3889


Checking for missing information and filling in

In [11]:
df_nan = df_gps[df_gps['Latitude'].isnull() | df_gps['Longitude'].isnull()]
df_nan


,PostalCode,Latitude,Longitude
76,M7R,NaN,NaN


Fetching geospatical data from csv provided by Coursera

In [12]:
import wget
#wget.download('https://cocl.us/Geospatial_data')
coord = pd.read_csv('Geospatial_Coordinates.csv')
print(coord[coord['Postal Code']=='M7R'])


   Postal Code   Latitude  Longitude
86         M7R  43.636966 -79.615819


Filling in missing information with csv provided by Coursera

In [13]:
plist=[]
for postalcode in df_nan['PostalCode']:
    lat = coord[coord['Postal Code']==postalcode]['Latitude'].values
    long = coord[coord['Postal Code']==postalcode]['Longitude'].values
    print(postalcode, lat, long)
    plist.append(postalcode)
    df_gps.loc[df_gps['PostalCode']==postalcode,'Latitude'] = lat
    df_gps.loc[df_gps['PostalCode']==postalcode,'Longitude'] = long

df_gps_check = df_gps[df_gps['PostalCode'].isin(plist)]


M7R [43.6369656] [-79.615819]


In [14]:
df_merge = pd.merge(df, df_gps, how='left', on='PostalCode')
df_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754500,-79.330000
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.664100,-79.388900
5,M9A,Etobicoke,Islington Avenue,43.666200,-79.528200
6,M1B,Scarborough,"Malvern , Rouge",43.811300,-79.193000
7,M3B,North York,Don Mills,43.745000,-79.359000
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706300,-79.309400
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657200,-79.378300


##  

## Part 3 - Explore and cluster the neighborhoods

Importing additional libraries

In [15]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Create a map of Toronto

In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'GEBOPUJNJ1I1BAO5U1FTND5P1Y4NRTMOKGN1K4UTDYBN2B00' # your Foursquare ID
CLIENT_SECRET = 'DKT3DQ5GSLCAB5NRDKUMHSRLNMNRB3DXF1QAZT2CTKAHKOVH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GEBOPUJNJ1I1BAO5U1FTND5P1Y4NRTMOKGN1K4UTDYBN2B00
CLIENT_SECRET:DKT3DQ5GSLCAB5NRDKUMHSRLNMNRB3DXF1QAZT2CTKAHKOVH


Let's restrict our data to the boroughs with the word 'Toronto' in them

In [19]:
df_toronto = df_merge[df_merge['Borough'].str.contains('Toronto')]
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
19,M4E,East Toronto,The Beaches,43.6784,-79.2941
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
24,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
25,M6G,Downtown Toronto,Christie,43.6683,-79.4205
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.6496,-79.3833
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.6655,-79.4378


Let's create a function to get the top 100 venues (by default) for all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the above function on each borough containing the word 'Toronto', setting the venues limit to 20 and create a new dataframe called *toronto_venues*.

In [21]:
LIMIT=20
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'],
                                   LIMIT=LIMIT
                                 )


Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(582, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park , Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park , Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park , Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check the venue categories

In [56]:
temp = toronto_venues['Venue Category'].sort_values().unique()
temp

array(['American Restaurant', 'Art Gallery', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'BBQ Joint',
       'Baby Store', 'Bakery', 'Bank', 'Bar', 'Baseball Field',
       'Basketball Stadium', 'Beach Bar', 'Beer Bar', 'Beer Store',
       'Bookstore', 'Brazilian Restaurant', 'Breakfast Spot', 'Brewery',
       'Bubble Tea Shop', 'Burger Joint', 'Burrito Place', 'Bus Line',
       'Butcher', 'Café', 'Candy Store', 'Caribbean Restaurant',
       'Cheese Shop', 'Chinese Restaurant', 'Church', 'Clothing Store',
       'Cocktail Bar', 'Coffee Shop', 'College Arts Building',
       'College Gym', 'College Theater', 'Comfort Food Restaurant',
       'Comic Shop', 'Concert Hall', 'Cosmetics Shop', 'Coworking Space',
       'Creperie', 'Cuban Restaurant', 'Dance Studio', 'Deli / Bodega',
       'Department Store', 'Dessert Shop', 'Diner', 'Distribution Center',
       'Dog Run', 'Dumpling Restaurant', 'Eastern European Restaurant',
       'Electronics Store', 'Even

Let's check how many venues were returned for each neighborhood

In [57]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton , Parkdale Village , Exhibition Place",20,20,20,20,20,20
Business reply mail Processing CentrE,16,16,16,16,16,16
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
Christie,12,12,12,12,12,12
Church and Wellesley,20,20,20,20,20,20
"Commerce Court , Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 159 uniques categories.


<a id='item3'></a>

## Analyze Each Neighborhood

In [59]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park , Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park , Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park , Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [60]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop('Neighborhood', axis=1, inplace=True)
toronto_onehot.head()

# add neighborhood column back to dataframe
#toronto_onehot['Total Number of venues'] = toronto_onehot[list(toronto_onehot.columns)].sum(axis=1)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + pd.Index.to_list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(20)

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [61]:
toronto_onehot.shape

(582, 159)

#### Next, let's group rows by neighborhood and filtering for neighborhood with number of venues less than the limit

In [62]:
toronto_onehot_filter = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_onehot_filter['Total venues'] = toronto_onehot_filter[list(toronto_onehot_filter.columns)].sum(axis=1)
toronto_onehot_filter = toronto_onehot_filter[toronto_onehot_filter['Total venues'] < LIMIT]
n_to_filter = list(toronto_onehot_filter['Neighborhood'])
n_to_filter

['Brockton , Parkdale Village , Exhibition Place',
 'Business reply mail Processing CentrE',
 'Central Bay Street',
 'Christie',
 'Davisville North',
 'Dufferin , Dovercourt Village',
 'Forest Hill North & West',
 'Harbourfront East , Union Station , Toronto Islands',
 'High Park , The Junction South',
 'Lawrence Park',
 'Moore Park , Summerhill East',
 'North Toronto West',
 'Rosedale',
 'Roselawn',
 'Studio District',
 'Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park',
 'The Beaches']

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped = toronto_grouped[~toronto_grouped['Neighborhood'].isin(n_to_filter)]
toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
3,"CN Tower , King and Spadina , Railway Lands , ...",0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.05,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05
6,Church and Wellesley,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,"Commerce Court , Victoria Hotel",0.05,0.00,0.00,0.00,0.0,0.00,0.0,0.05,0.0,...,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
8,Davisville,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11,"First Canadian Place , Underground city",0.05,0.00,0.00,0.00,0.0,0.00,0.0,0.05,0.0,...,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
13,"Garden District, Ryerson",0.00,0.05,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
16,"India Bazaar , The Beaches West",0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
17,"Kensington Market , Chinatown , Grange Park",0.00,0.00,0.05,0.00,0.0,0.00,0.0,0.05,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.05,0.00
19,"Little Portugal , Trinity",0.00,0.05,0.00,0.05,0.0,0.00,0.0,0.00,0.0,...,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.05


#### Let's confirm the new size

In [64]:
toronto_grouped.shape

(22, 159)

#### Let's print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0         Cocktail Bar  0.10
1                 Café  0.10
2         Liquor Store  0.05
3  Japanese Restaurant  0.05
4               Museum  0.05


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1                  Park  0.10
2              Beer Bar  0.05
3  Caribbean Restaurant  0.05
4             Speakeasy  0.05


----Church and Wellesley----
              venue  freq
0         Bookstore  0.05
1         Juice Bar  0.05
2      Burger Joint  0.05
3   Bubble Tea Shop  0.05
4  Ramen Restaurant  0.05


----Commerce Court , Victoria Hotel----
                 venue  freq
0                 Café  0.15
1          Coffee Shop  0.10
2           Restaurant  0.10
3   Seafood Restaurant  0.10
4  American Restaurant  0.05


----Davisville----
                venue  freq
0        Dessert Shop  0.10
1  Italian Restaurant  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Café,Park,Food Truck,Museum,Coffee Shop,Liquor Store,Pub,Restaurant,Japanese Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Italian Restaurant,Park,Yoga Studio,French Restaurant,Café,Ramen Restaurant,Hotel,Speakeasy,Brewery,Coffee Shop
6,Church and Wellesley,Gastropub,Bookstore,Creperie,Dance Studio,Gay Bar,Park,Diner,Pizza Place,Juice Bar,Pub
7,"Commerce Court , Victoria Hotel",Café,Coffee Shop,Restaurant,Seafood Restaurant,Greek Restaurant,Steakhouse,Bakery,Food Court,Gastropub,Gluten-free Restaurant
8,Davisville,Sandwich Place,Italian Restaurant,Dessert Shop,Indian Restaurant,Coffee Shop,Seafood Restaurant,Farmers Market,Fast Food Restaurant,Café,Sushi Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [68]:
toronto_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.00
3,"CN Tower , King and Spadina , Railway Lands , ...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.05
6,Church and Wellesley,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.05,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00
7,"Commerce Court , Victoria Hotel",0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,...,0.0,0.00,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.00
8,Davisville,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [69]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 2, 1, 2, 3, 0, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626,3,Coffee Shop,Breakfast Spot,Yoga Studio,Event Space,Restaurant,Electronics Store,Spa,Pub,Food Truck,Distribution Center
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889,0,Italian Restaurant,Park,Beer Bar,Mexican Restaurant,Diner,Distribution Center,College Theater,Ramen Restaurant,Café,Indian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3,Coffee Shop,Tea Room,Burrito Place,Sandwich Place,Electronics Store,Ramen Restaurant,Shopping Mall,Hotel,Plaza,Clothing Store
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,American Restaurant,Beer Bar,Cosmetics Shop,Middle Eastern Restaurant,Church,Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,1,Cocktail Bar,Café,Park,Food Truck,Museum,Coffee Shop,Liquor Store,Pub,Restaurant,Japanese Restaurant


In [71]:
toronto_merged.shape

(22, 16)

Finally, let's visualize the resulting clusters

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

Now, we can examine the neighborhoods in each cluster.

#### Cluster 1

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Queen's Park , Ontario Provincial Government",0,Italian Restaurant,Park,Beer Bar,Mexican Restaurant,Diner,Distribution Center,College Theater,Ramen Restaurant,Café,Indian Restaurant
37,"Little Portugal , Trinity",0,Bar,Yoga Studio,Record Shop,Art Gallery,Asian Restaurant,Beer Store,Brewery,Cocktail Bar,Cuban Restaurant,French Restaurant
47,"India Bazaar , The Beaches West",0,Park,Fast Food Restaurant,Burrito Place,Pizza Place,Sandwich Place,Ice Cream Shop,Brewery,Pub,Steakhouse,Fish & Chips Shop
75,"Parkdale , Roncesvalles",0,American Restaurant,Eastern European Restaurant,Sushi Restaurant,Dessert Shop,Pizza Place,Coffee Shop,Butcher,Fish Market,Italian Restaurant,Cuban Restaurant
99,Church and Wellesley,0,Gastropub,Bookstore,Creperie,Dance Studio,Gay Bar,Park,Diner,Pizza Place,Juice Bar,Pub


#### Cluster 2

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Berczy Park,1,Cocktail Bar,Café,Park,Food Truck,Museum,Coffee Shop,Liquor Store,Pub,Restaurant,Japanese Restaurant
74,"The Annex , North Midtown , Yorkville",1,Café,Sandwich Place,American Restaurant,History Museum,Mexican Restaurant,Park,Liquor Store,Pharmacy,Coffee Shop,Pizza Place
79,Davisville,1,Sandwich Place,Italian Restaurant,Dessert Shop,Indian Restaurant,Coffee Shop,Seafood Restaurant,Farmers Market,Fast Food Restaurant,Café,Sushi Restaurant
80,"University of Toronto , Harbord",1,Japanese Restaurant,Restaurant,Café,Bakery,Bookstore,Yoga Studio,Beer Bar,French Restaurant,Italian Restaurant,Dessert Shop
81,"Runnymede , Swansea",1,Café,Sushi Restaurant,Bookstore,Food,Falafel Restaurant,French Restaurant,Tea Room,Coffee Shop,Latin American Restaurant,Fish & Chips Shop
84,"Kensington Market , Chinatown , Grange Park",1,Café,Vietnamese Restaurant,Mexican Restaurant,Farmers Market,Cheese Shop,Coffee Shop,Dumpling Restaurant,Gaming Cafe,Bar,Bakery
87,"CN Tower , King and Spadina , Railway Lands , ...",1,Italian Restaurant,Park,Yoga Studio,French Restaurant,Café,Ramen Restaurant,Hotel,Speakeasy,Brewery,Coffee Shop
96,"St. James Town , Cabbagetown",1,Café,Restaurant,Thai Restaurant,Library,Coffee Shop,Caribbean Restaurant,Japanese Restaurant,Taiwanese Restaurant,Pet Store,Market


#### Cluster 3

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,St. James Town,2,Gastropub,Coffee Shop,Japanese Restaurant,Hotel,American Restaurant,Beer Bar,Cosmetics Shop,Middle Eastern Restaurant,Church,Restaurant
30,"Richmond , Adelaide , King",2,Café,Coffee Shop,Seafood Restaurant,Greek Restaurant,Steakhouse,Asian Restaurant,Concert Hall,Food Court,Gastropub,Gym / Fitness Center
42,"Toronto Dominion Centre , Design Exchange",2,Coffee Shop,American Restaurant,Beer Bar,Gym,Gym / Fitness Center,Hotel,Japanese Restaurant,Deli / Bodega,Concert Hall,Pizza Place
48,"Commerce Court , Victoria Hotel",2,Café,Coffee Shop,Restaurant,Seafood Restaurant,Greek Restaurant,Steakhouse,Bakery,Food Court,Gastropub,Gluten-free Restaurant
97,"First Canadian Place , Underground city",2,Café,Coffee Shop,Restaurant,Seafood Restaurant,Greek Restaurant,Steakhouse,Bakery,Food Court,Gastropub,Gluten-free Restaurant


#### Cluster 4

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park , Harbourfront",3,Coffee Shop,Breakfast Spot,Yoga Studio,Event Space,Restaurant,Electronics Store,Spa,Pub,Food Truck,Distribution Center
9,"Garden District, Ryerson",3,Coffee Shop,Tea Room,Burrito Place,Sandwich Place,Electronics Store,Ramen Restaurant,Shopping Mall,Hotel,Plaza,Clothing Store
92,Stn A PO Boxes,3,Hotel,Coffee Shop,New American Restaurant,Basketball Stadium,Salad Place,Japanese Restaurant,Plaza,Beer Bar,Sporting Goods Shop,Sports Bar


#### Cluster 5

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"The Danforth West , Riverdale",4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Spa,Bakery,Brewery,Bubble Tea Shop,Cosmetics Shop,Dessert Shop,Fruit & Vegetable Store
